In [1]:
from google.colab import drive
import os
import time
import pandas as pd
import numpy as np
import random

In [2]:
# Mounting Google Drive Location of training images
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/FourthBrain/CapstoneSamsungOCT/')    ###### You need to edit this line

Mounted at /content/drive


In [10]:
# Setting up the variables of where the processing will occur
home_path = "/content/drive/MyDrive/FourthBrain/CapstoneSamsungOCT/"    ###### You need to edit this line
sinGAN_dir = home_path + "sinGAN-tf/"
data_dir = home_path + "Data/OCT2017/"
synthetic_dir = data_dir + "sinGAN-Data/synthetic/"
sinGAN_model = sinGAN_dir + "main.py"

# You just need to set the class you are training for, set it to all CAPITAL
train_class = "CNV"     ######  Edit this line
sinGAN_train_dir = data_dir + "sinGAN-Data/" + train_class
person_training = home_path + "Bashar/"    #####  Edit this line, I created the directories
sinGAN_chkpt = person_training + "training_checkpoints/"
os.chdir(person_training)

#  Change this value to True only if you have new images that you want to train on, otherwise leave to FALSE
REGEN = False

In [4]:
sinGAN_model

'/content/drive/MyDrive/FourthBrain/CapstoneSamsungOCT/sinGAN-tf/main.py'

In [5]:
def create_df (path, sub_dir):
  df = pd.DataFrame(columns=['filename', 'cnv', 'dme', 'drusen', 'normal'])
  condition = {'normal': 0, 'cnv': 0, 'dme':0, 'drusen': 0}
  files = os.listdir(path + sub_dir)
  if (sub_dir == 'NORMAL'):
    condition['normal'] = 1
  elif (sub_dir == 'CNV'):
    condition['cnv'] = 1
  elif (sub_dir == 'DME'):
    condition['dme'] = 1
  else:
    condition['drusen']= 1
  for f in files:
    df = df.append({'filename': path +  sub_dir  + "/" + f,  
                    'cnv': condition['cnv'],
                    'dme': condition['dme'],
                    'drusen': condition['drusen'],
                    'normal': condition['normal']}, ignore_index=True)
  return df

In [6]:
classes = ['NORMAL', 'CNV', 'DME', 'DRUSEN']
cols = [x.lower() for x in classes]
label = {0: 'CNV', 1: 'DME', 2: 'DRUSEN', 3: 'NORMAL'}

if (REGEN):
  for directory in classes:
    train_df = create_df(data_dir + "sinGAN-Data/", directory)
    train_df.set_index('filename', inplace=True)
    train_df.to_csv(data_dir + "sinGAN-Data/" + directory + "_train.csv")
else:
  # Change this line if you are going to resume training from the last file you left off
  train_df = pd.read_csv(sinGAN_train_dir + "_train.csv")

In [7]:
def train_sinGAN(filename):
  !python $sinGAN_model train --image $filename --num_scales 8

In [8]:
def pick_random(filename):
  !python $sinGAN_model inference --image $filename  --dir $sinGAN_chkpt  --mode random_sample  --inject_scale 0
  rand_num = random.sample(range(0,49),20)
  print (rand_num)
  for i in rand_num:
    os.replace("./results/random_sample/random_sample_" + str(i) + ".jpg", synthetic_dir + train_class + "/" + str(i) + "_"  + os.path.basename(filename))

In [ ]:
remainingDF = train_df.copy(True)

In [13]:
for row in train_df["filename"]:
  train_sinGAN("." + row)
  pick_random("." + row)
  row_index = remainingDF[remainingDF['filename']==row].index
  remainingDF.drop(row_index, inplace=True)
  remainingDF.to_csv(train_class + "_remaining.csv")

Directory ./training_checkpoints already exists
2021-08-23 14:13:18.082688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 14:13:18.090587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 14:13:18.091252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 14:13:18.092145: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild

KeyError: ignored